# Transportation Problem
Consider a trucking company A that is providing services in 26 different routes. Each routes has impact (net profit/loss in terms of dollars of running a truck on that route), minimum and maximum volume constraints that trucks can handel in those routes.The trucking company also has a limit on how many miles the trucks could cover in a week due to a finite amount of trucks ( more than 20000 miles but less than 22000 miles ). The goal is to optimally select which lanes to run and at what volumes to maximize equipment utilization and financial impact.

In [3]:
# install pandas
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/df/92/a3fa053c74198f9f0224b2c04dc74f41d2e14e30329c082f7a657f9ca4c5/pandas-2.1.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ---------------------- ----------------- 194.6/341.8 kB ? eta -:--:--
     -------------------------------------- 341.8/341.8 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
    --------------------------------------- 0.3/10.5 MB 5.2 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/10.5 MB 7.4 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.5 MB 8.9 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/10.5 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install openpyxl

     ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
     - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
     --------- --------------------------- 61.4/250.0 kB 656.4 kB/s eta 0:00:01
     -------------------------- ----------- 174.1/250.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 1.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# import pandas and pulp
import pandas as pd
import pulp
from pulp import *

In [13]:
#Import the data, set the laneId as the Index
data = pd.read_excel('C:/Users/User/OneDrive/Documents/OR PROJECTS/SampleData.xlsx')

In [14]:
data

,LaneId,Distance,ImpactPer,MinVol,MaxVol
0,0,570,349.50,2,6
1,1,117,109.42,0,3
2,2,120,0.45,15,25
3,3,1116,-915.59,0,0
4,4,1058,-1000.56,0,0
5,5,942,-550.68,0,0
6,6,1103,-960.45,0,0
7,7,921,-787.00,0,2
8,8,1091,-927.75,0,0
9,9,443,82.08,3,8


In [1]:
# setting limits on distance covered by trucks
lower_limit = 20000
upper_limit = 22000

In [21]:
# Creating Lp model
model = pulp.LpProblem('RouteOptimization', LpMaximize)
 
# Getting values for lanes, max-min volume, cost-impact and distance
Lanes = data.index
MaxVols = data['MaxVol']
MinVols = data['MinVol']
Impacts = data['ImpactPer']
Miles = data['Distance']

# creating variables
x = LpVariable.dicts('Lane', Lanes,cat= LpInteger)
 
for l in Lanes:
 x[l].bounds(MinVols[l], MaxVols[l])


In [17]:
x

{0: Lane_0,
 1: Lane_1,
 2: Lane_2,
 3: Lane_3,
 4: Lane_4,
 5: Lane_5,
 6: Lane_6,
 7: Lane_7,
 8: Lane_8,
 9: Lane_9,
 10: Lane_10,
 11: Lane_11,
 12: Lane_12,
 13: Lane_13,
 14: Lane_14,
 15: Lane_15,
 16: Lane_16,
 17: Lane_17,
 18: Lane_18,
 19: Lane_19,
 20: Lane_20,
 21: Lane_21,
 22: Lane_22,
 23: Lane_23,
 24: Lane_24,
 25: Lane_25}

In [23]:
# Creating objective function

model += sum([x[l] * Impacts[l] for l in Lanes])

# Creating constraints
prob += lpSum([Miles[l] * x[l] for l in Lanes]) <= upper_limit
prob += lpSum([Miles[l] * x[l] for l in Lanes]) >= lower_limit

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [24]:
prob.solve()

1

In [46]:
# results

print('Total cost = ',model.objective.value())


Total cost =  938.8299999999998


In [55]:
# volume of trucks in each lane
for l in Lanes:
    print(f"Lane {l}",' : ',x[l].varValue)

Lane 0  :  2.0
Lane 1  :  0.0
Lane 2  :  15.0
Lane 3  :  0.0
Lane 4  :  0.0
Lane 5  :  0.0
Lane 6  :  0.0
Lane 7  :  1.0
Lane 8  :  0.0
Lane 9  :  3.0
Lane 10  :  0.0
Lane 11  :  2.0
Lane 12  :  8.0
Lane 13  :  0.0
Lane 14  :  0.0
Lane 15  :  0.0
Lane 16  :  0.0
Lane 17  :  0.0
Lane 18  :  1.0
Lane 19  :  0.0
Lane 20  :  0.0
Lane 21  :  6.0
Lane 22  :  0.0
Lane 23  :  1.0
Lane 24  :  4.0
Lane 25  :  0.0
